# Missile Geometry
#### Composed of 5 milestones

In [189]:
"""
**************************************************************************
*
* Author: Cooper Wolf
* Email: ctwolf1014@my.msutexas.edu
* Label: P01
* Title: Missile Geometry
* Course: CMPS 5993
* Semester: Spring 2026
*
* Description:
* This noteboook visualizes the geometry of a missile's range using Folium and GeoPandas.
* It displays a world map with country boundaries and marks the base location from which the missile is launched.
*
**************************************************************************
"""

"\n**************************************************************************\n*\n* Author: Cooper Wolf\n* Email: ctwolf1014@my.msutexas.edu\n* Label: P01\n* Title: Missile Geometry\n* Course: CMPS 5993\n* Semester: Spring 2026\n*\n* Description:\n* This noteboook visualizes the geometry of a missile's range using Folium and GeoPandas.\n* It displays a world map with country boundaries and marks the base location from which the missile is launched.\n*\n**************************************************************************\n"

## Milestone 1 - Plot the World

##### Tasks: Load a world countries geojson
#####        Display using folium
#####        Add base location as point marker

In [190]:
# Installing libraries
#%pip install folium geopandas

In [191]:
# Importing libraries
import geopandas as gpd
import folium
from __future__ import annotations
import math
from typing import Tuple
import json
from folium.features import DivIcon
from shapely.geometry import Point
from shapely.geometry import LineString
import pandas as pd

### Load world geojson

In [192]:
# Reading the world data
world = gpd.read_file("json/countries.geojson")

# Creating a map
m = folium.Map(location=[20,0], zoom_start=2)

### Display using folium

In [193]:
# Converting GeoDataFrame to GeoJSON and adding to Folium
folium.GeoJson(world, name="World Countries").add_to(m)

### Adding base location as point marker

In [194]:
# Adding base location as a point marker
base_lat = 38.89767
base_lon = -77.03655

# Adding a marker for the base location
folium.Marker(location=[base_lat, base_lon], popup="Base", icon=folium.Icon(color="red", icon="home")).add_to(m)

## Milestone 2 - Distance & Bearing

##### Task: Compute distance from each threat origin to base
#####       Identify closest threat
#####       Display threat origin as points

#### Helper functions graciously provided by Dr. G

In [195]:
# ----------------------------------------
# Constants
# ----------------------------------------
EARTH_RADIUS_KM = 6371.0088  # mean Earth radius


# ----------------------------------------
# Distance: Haversine (km)
# ----------------------------------------
def haversine_km(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Compute great-circle distance between two lat/lon points.
    Returns distance in kilometers.
    """
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)

    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)

    a = (
        math.sin(dphi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(dlmb / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return EARTH_RADIUS_KM * c


# ----------------------------------------
# Destination point (bearing + distance)
# ----------------------------------------
def destination_point(
    lat: float, lon: float, bearing_deg: float, distance_km: float
) -> Tuple[float, float]:
    """
    Compute destination lat/lon given:
      - start lat/lon
      - bearing in degrees
      - distance in km

    Uses great-circle formula.
    """
    bearing = math.radians(bearing_deg)

    lat1 = math.radians(lat)
    lon1 = math.radians(lon)
    d = distance_km / EARTH_RADIUS_KM

    lat2 = math.asin(
        math.sin(lat1) * math.cos(d) + math.cos(lat1) * math.sin(d) * math.cos(bearing)
    )

    lon2 = lon1 + math.atan2(
        math.sin(bearing) * math.sin(d) * math.cos(lat1),
        math.cos(d) - math.sin(lat1) * math.sin(lat2),
    )

    # normalize longitude to [-180, 180]
    lon2 = (math.degrees(lon2) + 540) % 360 - 180
    lat2 = math.degrees(lat2)

    return lat2, lon2

### Computing distance from each threat to base
### Identifing closest threats

In [196]:
## Read in threats, calculate each distance from base, and identify closest threat, and display their original location as points


# Read in threats from JSON file
with open("json/threats.json", "r") as f:
    threats = json.load(f)
    
# Calculate distance from base for each threat and store in dictionary
for threat in threats:
    threat["dist_from_base"] = haversine_km(base_lat, base_lon, threat["origin_lat"], threat["origin_lon"])
    
# Printing threat and its distance from base
print("Threat distances from base:")
for threat in threats:
    print(f"Threat ID: {threat["id"]}, Distance from base: {threat["dist_from_base"]:.2f} km")

# Identify closest threat    
threats_ordered_by_distance = sorted(threats, key=lambda x: x["dist_from_base"])
print("\nThreats ordered by distance from base:")

for threat in threats_ordered_by_distance:
    print(f"Threat ID: {threat["id"]}, Distance from base: {threat["dist_from_base"]:.2f} km")


Threat distances from base:
Threat ID: T001, Distance from base: 2512.64 km
Threat ID: T002, Distance from base: 4880.88 km
Threat ID: T003, Distance from base: 4703.22 km
Threat ID: T004, Distance from base: 4874.54 km
Threat ID: T005, Distance from base: 2402.38 km
Threat ID: T006, Distance from base: 2879.01 km
Threat ID: T007, Distance from base: 4806.17 km
Threat ID: T008, Distance from base: 3957.28 km
Threat ID: T009, Distance from base: 2328.06 km
Threat ID: T010, Distance from base: 3626.03 km

Threats ordered by distance from base:
Threat ID: T009, Distance from base: 2328.06 km
Threat ID: T005, Distance from base: 2402.38 km
Threat ID: T001, Distance from base: 2512.64 km
Threat ID: T006, Distance from base: 2879.01 km
Threat ID: T010, Distance from base: 3626.03 km
Threat ID: T008, Distance from base: 3957.28 km
Threat ID: T003, Distance from base: 4703.22 km
Threat ID: T007, Distance from base: 4806.17 km
Threat ID: T004, Distance from base: 4874.54 km
Threat ID: T002, Dis

### Displaying threat origins as points

In [197]:
# Displaying threat origins as points on the map
for threat in threats:
    
    # Assigning icons based on threat type
    if threat["type"] == "alien":
        icon_html = '<div style="font-size: 24px;">👽</div>'
    elif threat["type"] == "kaiju":
        icon_html = '<div style="font-size: 24px;">🦖</div>'
    elif threat["type"] == "airborne":
        icon_html = '<div style="font-size: 24px;">✈️</div>'
    elif threat["type"] == "orbital":
        icon_html = '<div style="font-size: 24px;">🛰️</div>'

# Adding markers for each threat origin with appropriate icons
    folium.Marker(
        location=[threat["origin_lat"], threat["origin_lon"]],
        popup=f'{threat["id"]} - {threat["type"]}',
        icon=DivIcon(html=icon_html)
    ).add_to(m)

## Milestone 3 - Trajectories

##### Tasks: For each threat: compute distination point after a fixed time interval
#####                         generate intermediate points
#####                         construct a LineString trajectory
#####        Plot trajectories on the map

In [198]:
# Function  to generate intermediate points
def generate_points(lat1: float, lon1: float, lat2: float, lon2: float, num_points: int) -> list[Tuple[float, float]]:
    """
    Generate intermediate points between two lat/lon coordinates.
    Returns a list of (lat, lon) tuples.
    """
    
    # Create points list
    points = []
    
    # Generate intermediate points using linear interpolation
    for i in range(1, num_points + 1):
        fraction = i / (num_points + 1)
        lat = lat1 + (lat2 - lat1) * fraction
        lon = lon1 + (lon2 - lon1) * fraction
        points.append((lat, lon))
    return points

In [199]:
# Dictionary to store threat id and its destination point
threat_points = {}

# Looping through each threat
for threat in threats:
    threat["dest_lat"], threat["dest_lon"] = destination_point(threat["origin_lat"], threat["origin_lon"], threat["bearing_deg"], threat["dist_from_base"])
    
    # Printing destination points
    print(f"Threat ID: {threat["id"]}, Destination Lat: {threat["dest_lat"]:.4f}, Destination Lon: {threat["dest_lon"]:.4f}")
    
    # Generate intermediate
    threat_points[threat["id"]] = generate_points(threat["origin_lat"], threat["origin_lon"], threat["dest_lat"], threat["dest_lon"], num_points=8)
    

Threat ID: T001, Destination Lat: 36.8800, Destination Lon: -92.5684
Threat ID: T002, Destination Lat: 50.5944, Destination Lon: -93.8280
Threat ID: T003, Destination Lat: 40.5373, Destination Lon: -78.8787
Threat ID: T004, Destination Lat: 34.7698, Destination Lon: -74.2756
Threat ID: T005, Destination Lat: 22.2223, Destination Lon: -92.3439
Threat ID: T006, Destination Lat: 27.0437, Destination Lon: -98.0965
Threat ID: T007, Destination Lat: 23.9547, Destination Lon: -80.3620
Threat ID: T008, Destination Lat: 22.5100, Destination Lon: -91.7222
Threat ID: T009, Destination Lat: 37.9031, Destination Lon: -91.5162
Threat ID: T010, Destination Lat: 28.1274, Destination Lon: -89.5994


#### Plotting trajectories on map

In [200]:
# Creating LineString / trajectory for each threat
for threat in threats:
    
    # Default color
    color = "red"
    
    # Assign color based on type
    if threat["type"] == "alien":
        color = "green"
    elif threat["type"] == "kaiju":
        color = "blue"
    elif threat["type"] == "airborne":
        color = "orange"
    elif threat["type"] == "orbital":
        color = "purple"
    
    # Get intermediate points for this threat
    points = threat_points[threat["id"]]
    
    # Plot each intermediate point
    for point in points:
        folium.CircleMarker(location=point, radius=3, color=color, fill=True).add_to(m)
        
    # Adding starting and ending lat lon to the points list for trajectory line
    points.insert(0, (threat["origin_lat"], threat["origin_lon"]))
    points.append((threat["dest_lat"], threat["dest_lon"]))
    
    # Plotting destination point as red marker
    #folium.CircleMarker(location=(threat["dest_lat"], threat["dest_lon"]), radius=4, color="red", fill=True).add_to(m)
    
    # Draw the trajectory line using the same points
    folium.PolyLine(points, color=color, weight=2).add_to(m)


## Milestone 4 - Intersects & Borders

##### Tasks: Determine which country each trajectory intersects
#####        Determine a trajectory passes within a threshold distance of your base
#####        Highlight intersected countries on the map

#### Determining which country are intersected by threats

In [201]:
# Dict to store countries that are intersected by each threat trajectory
countries_intersected = {}

# Loop through each threat
for threat in threats:
    
    # Build list of corrected (lon, lat) points
    corrected_points = [(lon, lat) for (lat, lon) in threat_points[threat["id"]]]
    
    # Create full trajectory line
    trajectory = LineString(corrected_points)
    
    # Check intersections
    intersected = world[world.intersects(trajectory)]
    
    # Store all intersected countries as a list
    countries_intersected[threat['id']] = list(intersected["ADMIN"])

# print intersected countries
print("\nSummary of all threats:")
for threat_id, countries in countries_intersected.items():
    print(f"Threat {threat_id} intersects {countries}")



Summary of all threats:
Threat T001 intersects ['Canada', 'United States of America']
Threat T002 intersects ['Canada', 'Mexico', 'United States of America']
Threat T003 intersects ['Mexico', 'United States of America']
Threat T004 intersects ['Mexico', 'United States of America']
Threat T005 intersects ['Dominican Republic', 'Haiti', 'Mexico']
Threat T006 intersects ['Mexico', 'United States of America']
Threat T007 intersects ['Mexico', 'United States of America']
Threat T008 intersects ['Mexico', 'United States of America']
Threat T009 intersects ['Canada', 'United States of America']
Threat T010 intersects ['Canada', 'United States of America']


#### Highlight intersected countries on map

In [202]:
# List of countries to highlight
countries_to_highlight = []

# Loop through intersected countries
for threat_id, countries in countries_intersected.items():
    for country in countries:
        if country not in countries_to_highlight:
            countries_to_highlight.append(country)
        
highlighted_countries = world[world["ADMIN"].isin(countries_to_highlight)]

# Add highlighted countries to the map
folium.GeoJson(
    highlighted_countries,
    style_function=lambda feature: {
        "fillColor": "yellow",       # Highlight color
        "color": "black",         # Border color
        "weight": 2,              # Border thickness
        "fillOpacity": 0.1        # Transparency
    }).add_to(m)

#### Determining which threats pass threshold of base

In [203]:
# Base threshold for threats km
base_threshold = 300
base = Point(base_lon, base_lat)
radius_in_degrees = base_threshold / 111
danger_zone = base.buffer(radius_in_degrees)

# Store threats in danger zone
threat_dangerzone = []

# Loop through each threat
for threat in threats:
    
    # Printing threat id if threat is within danger zone
    for point in threat_points[threat["id"]]:

        # Get lon and lat of threat at each intermediate point
        location = Point(point[1], point[0])
        
        # Determine if threat is withing dangerzone at specific point
        if location.within(danger_zone) and threat["id"] not in threat_dangerzone:
            threat_dangerzone.append(threat["id"])
            print(threat["id"], "Is within danger zone!")


T003 Is within danger zone!


In [204]:
# Plotting threshold on map
folium.Circle(
    location=[base_lat, base_lon],
    radius=base_threshold * 1000,  # km to meters
    color="blue",
    fill_opacity=0.2,
    popup="Base Danger Zone"
).add_to(m)

## Milestone 5 - Damage Zones

##### Tasks: Create buffer zone around each trajectory endpoint
#####        Buffer size depends on threat type
#####        Determine which countries fall within damage zones

In [205]:
# Dict to store countries damaged
countries_damaged = {}


# Assigning buffer zone for each threat type
for threat in threats:    
    if threat["type"] == "alien":
        buffer = 300
    elif threat["type"] == "kaiju":
        buffer = 300
    elif threat["type"] == "airborne":
        buffer = 400
    elif threat["type"] == "orbital":
        buffer = 500
        
    # Create buffer zone around each threats destination point
    buffer_pt = Point(threat["dest_lon"], threat["dest_lat"])
    radius_in_degrees = buffer / 111
    damage_zone = buffer_pt.buffer(radius_in_degrees)
    
    # Plotting damage raduis of each threat
    folium.Circle(
        location=[threat["dest_lat"], threat["dest_lon"]],
        radius=buffer * 1000,  # km to meters
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.2,
        popup="Danger Zone"
    ).add_to(m)
    
    # Store countries affected by threats danger zones
    damaged = world[world.intersects(damage_zone)]
    
    # Store severity results
    countries_damaged[threat["id"]] = []

    # Loop through each country and calculate severity
    for idx, country in damaged.iterrows():
        
        # Calculate intersection between country and damage zone
        intersection = country.geometry.intersection(damage_zone)
        
        # Calculate severity of damage based on country area
        if not intersection.is_empty:
            damage_area = intersection.area
            country_area = country.geometry.area
            
            # Calculate severity percentage
            severity = round((damage_area / country_area) * 100,3)
            
            # Store country, severity, and type of threat in the countries_damaged dictionary
            countries_damaged[threat["id"]].append({
                "country": country["ADMIN"],
                "severity": severity,
                "type": threat["type"]
        })


#### Create table of countries damaged, threat type, and serverity

In [206]:
# list to store rows
rows = []

# Loop through countries_damaged dict to create rows for DF
for threat_id, entries in countries_damaged.items():
    for entry in entries:
        
        # Append row to list
        rows.append({
            "Threat ID": threat_id,
            "Country": entry["country"],
            "Threat Type": entry["type"],
            "Severity (%)": entry["severity"]
        })

# Create DataFrame from rows
df = pd.DataFrame(rows)

# Printing DataFrame
print(df)

# Exporting DataFrame to CSV
df.to_csv("csv/threat_impact_summary.csv", index=False)

   Threat ID                   Country Threat Type  Severity (%)
0       T001  United States of America       alien         2.052
1       T002                    Canada       alien         1.220
2       T002  United States of America       alien         0.203
3       T003                    Canada       alien         0.050
4       T003  United States of America       alien         1.974
5       T004  United States of America       kaiju         0.101
6       T005                    Mexico       kaiju         0.158
7       T006                    Mexico       alien         4.131
8       T006  United States of America       alien         0.841
9       T007               The Bahamas       alien        35.852
10      T007                      Cuba       alien        37.658
11      T007  United States of America       alien         0.161
12      T008                    Mexico    airborne         1.912
13      T009  United States of America       alien         2.052
14      T010  United Stat

#### Creating Legend

In [207]:
legend_html = """
<div style="
position: fixed; 
bottom: 50px; left: 50px; width: 220px; height: 250px; 
background-color: white;
border:2px solid grey; 
z-index:9999; 
font-size:14px;
padding: 10px;
">
<b>Legend</b><br><br>

<span style="color:red;">🔴</span> Threat Danger Zone<br>
<span style="color:red;">👽</span> Alien (300 km)<br>
<span style="color:orange;">🦖</span> Kaiju (300 km)<br>
<span style="color:purple;">✈️</span> Airborne (400 km)<br>
<span style="color:black;">🛰️</span> Orbital (500 km)<br>
<span style="color:blue;">🔵</span> Base Zone (300 km)<br>
<i class="fa fa-home" style="margin-right:5px;"></i> Base Location

</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

#### Saving map

In [208]:
m.save("html/milestone5_map.html")